In [16]:
import pandas as pd 
import dask.dataframe as dd
import numpy as np 
from pyliftover import LiftOver

#Declare file locations
root = "/Users/npatel/Desktop/Dr.Chen_Lab/Neel_Data_Results/Task3-Gwas_data"
#import data from files as dataframe
GWAS_data = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Data/meta3.INTsnplist_F_2_hg18.txt", sep = '\s+' )
snp126 = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Genome_Data/snp126.txt", sep = '\s+',names = ['chrom', 'chromStart' ,'chromEnd','SNP'], usecols = [1,2,3,4])


GWAS_data

,CHR,SNP,BP,A1,A2,P,BETA
0,1.0,rs12565286,711153.0,C,G,0.7964,0.0236
1,1.0,rs11804171,713682.0,A,T,0.7976,0.0234
2,1.0,rs2977670,713754.0,C,G,0.8001,-0.0231
3,1.0,rs12138618,740098.0,A,G,0.7316,0.0585
4,1.0,rs3094315,742429.0,A,G,0.1312,-0.0541
...,...,...,...,...,...,...,...
2821729,NaN,rs9999872,NaN,A,G,0.6472,-0.1220
2821730,NaN,rs9999880,NaN,A,G,0.9726,0.0110
2821731,NaN,rs9999884,NaN,T,C,0.6525,-0.1690
2821732,NaN,rs9999929,NaN,T,G,0.6068,-350.4600


In [17]:
#compare chrom data to GWAS data by RS#
merge1 = pd.merge(GWAS_data, snp126, on = "SNP",  how='outer', indicator=True)

Updated_GWAS = merge1.loc[merge1["_merge"] == "both"].drop("_merge", axis=1)

#RSname matched with 
merge1


,CHR,SNP,BP,A1,A2,P,BETA,chrom,chromStart,chromEnd,_merge
0,1.0,rs12565286,711153.0,C,G,0.7964,0.0236,chr1,711152,711153,both
1,1.0,rs11804171,713682.0,A,T,0.7976,0.0234,chr1,713681,713682,both
2,1.0,rs2977670,713754.0,C,G,0.8001,-0.0231,chr1,713753,713754,both
3,1.0,rs12138618,740098.0,A,G,0.7316,0.0585,chr1,740097,740098,both
4,1.0,rs3094315,742429.0,A,G,0.1312,-0.0541,chr1,742428,742429,both
...,...,...,...,...,...,...,...,...,...,...,...
12351936,NaN,rs28462370,NaN,NaN,NaN,NaN,NaN,chrY,57442439,57442440,right_only
12351937,NaN,rs28607775,NaN,NaN,NaN,NaN,NaN,chrY,57442478,57442479,right_only
12351938,NaN,rs28476519,NaN,NaN,NaN,NaN,NaN,chrY,57442486,57442487,right_only
12351939,NaN,rs28628009,NaN,NaN,NaN,NaN,NaN,chrY,57442497,57442498,right_only


In [27]:

final = merge1[['chrom', 'chromStart', 'chromEnd', 'SNP']]

#Set up new data frame with a single string to be BED formate for liftover tool
exportList = final['chrom'].astype(str) + ' ' + final['chromStart'].astype(str) + ' ' + final['chromEnd'].astype(str)
final

final['chromEnd'] = final['chromEnd'].astype(int)
#Export to txt for liftover
exportList.to_csv(root + '/' + "GWASexportLiftList(hg18).txt", index = False,)
final['hg19'] = np.nan
final.dtypes

/var/folders/69/9n6tch_943x7rcbhpz476s8c0000gn/T/ipykernel_97895/1720840698.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['chromEnd'] = final['chromEnd'].astype(int)
/var/folders/69/9n6tch_943x7rcbhpz476s8c0000gn/T/ipykernel_97895/1720840698.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['hg19'] = np.nan


chrom          object
chromStart      int64
chromEnd        int64
SNP            object
hg19          float64
dtype: object

In [28]:
lo = LiftOver('hg18', 'hg19')

for column in final: 
    final.["hg19"] = lo.convert_coordinate(final.chrom,final.chromEnd)


final

SyntaxError: invalid syntax (3257780842.py, line 4)